Import the pre trained model

In [1]:
import tensorflow as tf
import tensorflow.keras

2022-10-25 10:31:50.797679: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
model = tf.keras.models.load_model('/datasets/nicolas_facchinetti/trained_models/EmoDB/firstCNN.h5')
model.summary()

2022-10-25 10:31:51.828476: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-25 10:31:51.829267: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-25 10:31:51.866244: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-10-25 10:31:51.866284: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-10-25 10:31:51.869929: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-10-25 10:31:51.870014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-1

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

Import the pre processed data

In [3]:
import numpy as np
x_train = np.load('/datasets/nicolas_facchinetti/processed_data/EmoDB/first_experiment/x_im_train.npy')
x_test = np.load('/datasets/nicolas_facchinetti/processed_data/EmoDB/first_experiment/x_im_test.npy')
y_train = np.load('/datasets/nicolas_facchinetti/processed_data/EmoDB/first_experiment/y_im_train.npy')
y_test = np.load('/datasets/nicolas_facchinetti/processed_data/EmoDB/first_experiment/y_im_test.npy')

In [4]:
_, accuracy_test_original = model.evaluate(x_test, y_test)
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))

2022-10-19 17:59:54.323549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-19 17:59:54.339415: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3301490000 Hz
2022-10-19 17:59:54.459858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-10-19 17:59:54.686435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


4/4 [==============================] - 1s 14ms/step - loss: 0.9659 - accuracy: 0.6637
Accuracy on original test data: 66.37%


Make ART model

In [5]:
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, CarliniL0Method

/home/nicolas/.conda/envs/thesis/lib/python3.9/site-packages/art/estimators/certification/__init__.py:13: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [6]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
classifier = TensorFlowV2Classifier(model=model, nb_classes=7, input_shape=(256, 256, 3), loss_object=loss_object)

In [7]:
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.3)

In [9]:
x_test_adv = attack_fgsm.generate(x_test)

In [10]:
_, accuracy_test_attack = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))

print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test_attack * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))

4/4 [==============================] - 0s 8ms/step - loss: 24.4051 - accuracy: 0.1062
Accuracy on adversarial test data: 10.62%
Average perturbation: 0.24


In [10]:
attack_cw = CarliniL0Method(classifier=classifier)

x_test_adv = attack_cw.generate(x_test)


KeyboardInterrupt



In [ ]:
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))